# Data

In [ ]:
!pip install pyvi
!pip install transformers
!pip install underthesea
!pip install sentence_transformers
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 31.0 MB/s 
     |████████████████████████████████| 1.0 MB 66.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 11.8 MB/s 
     |████████████████████████████████| 182 kB 62.9 MB/s 
     |████████████████████████████████| 7.6 MB 12.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 19.6 MB/s 
     |████████████████████████████████| 591 kB 63.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 1.3 MB 58.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3

In [ ]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from pyvi import ViTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import pickle
from sentence_transformers import SentenceTransformer, util
tqdm.pandas()
import json
import re
from transformers import pipeline
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from scipy import sparse
import sklearn
import underthesea
import os
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from underthesea import text_normalize
from sklearn.decomposition import TruncatedSVD
import math
import torch
import sklearn
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# !wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
# !wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
# !wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-private_test_v3.zip
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
# !unzip -q /content/e2eqa-train+public_test-v1.zip
# !unzip -q /content/wikipedia_20220620_cleaned.zip
# !unzip -q /content/e2eqa-private_test_v3.zip

--2023-01-01 15:48:27--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords-dash.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0s      

2023-01-01 15:48:28 (89.9 MB/s) - ‘vietnamese-stopwords-dash.txt’ saved [20475/20475]



In [ ]:
# data_file = open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json')
# data = json.load(data_file)
stopword_file = open('vietnamese-stopwords-dash.txt', 'r')
stopword = stopword_file.read()
stopword = stopword.split('\n')

# ModelForQuestionAnswering

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-large")
model = AutoModelForQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-large")

qa_model = pipeline("question-answering", model=model,tokenizer=tokenizer,device=0)


# "hogger32/xlmRoberta-for-VietnameseQA", "nguyenvulebinh/vi-mrc-base"

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
def remove_stopword(lsword):
  return lsword
  res = []
  for word in lsword.split():
    if not word in stopword:
      res.append(word) 
  return res

#QA_model

In [ ]:
def normalize_text(text):
    from bs4 import BeautifulSoup
    import re
    def strip_html_tags(text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text
    text = strip_html_tags(text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove = if more than 1
    text = re.sub(r'={2,}', ' ', text)
    # emove duplicate spaces
    text = re.sub(r'  +', ' ', text)
    # remove duplicate newline characters if more than 2
    text = re.sub(r'\n+', '. ', text)

    # tokenize_text = tokenize(text)
    return text

def remove_stopword(lsword):
  return lsword

In [ ]:
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

# def get_gold_answers(example):
#     """helper function that retrieves all possible true answers from a squad2.0 example"""
    
#     gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

#     # if gold_answers doesn't exist it's because this is a negative example - 
#     # the only correct answer is an empty string
#     if not gold_answers:
#         gold_answers = [""]
        
#     return gold_answers

## **BM25**

In [ ]:
data_file = open('/content/train_reg.json', 'r', encoding='utf-8-sig')
data = json.load(data_file)['data']

In [ ]:
paragraphs = []
questions = []
for row in data:
    for paragraph in row['paragraphs']:
        paragraphs.append(paragraph['context'])
        for qa in paragraph['qas']:
            questions.append((qa['question'], qa['answers'][0]['text']))

In [ ]:
from underthesea import sent_tokenize
from tqdm import tqdm
def segment_documents(docs, max_length=6, stride=3):
    """
    Segment documents into smaller chunks
    """
    print('Segmenting documents into smaller chunks...')
    segments = []
    for doc in tqdm(docs):
        sentences = sent_tokenize(doc)
        for i in range(0, len(sentences), stride):
            segment = ' '.join(sentences[i:i+max_length])
            segments.append(segment)
    print('Number of documents:', len(docs))
    print('Number of segments:', len(segments))
    return segments

In [ ]:
corpus_split_df=segment_documents(paragraphs)

Segmenting documents into smaller chunks...


100%|██████████| 75/75 [00:00<00:00, 1249.11it/s]

Number of documents: 75
Number of segments: 122


In [ ]:
# !pip install rank_bm25

In [ ]:
# from rank_bm25 import BM25Okapi

# tokenized_corpus = [doc.split(" ") for doc in corpus_split_df]
# bm25 = BM25Okapi(tokenized_corpus)

# def get_bm25_score(query, corpus_split_df, bm25):
#     tokenized_query = query.split(" ")
#     doc_scores = bm25.get_scores(tokenized_query)
#     return doc_scores

In [ ]:
def combine_metadata_wiki(item):
    newItem = {}
    # newItem['title'] = item['title']
    # newItem['text'] = item['text']
    newItem['text'] = item
    return newItem
def count_word_in_dataset_wiki(items):
    '''
    count number of word in dataset
    '''
    nameCounter = {}
    descCounter = {}
    for item in items:
        # for word in item['unigram_title'].keys():
        #     if word in nameCounter.keys():
        #         nameCounter[word] += 1
        #     else:
        #         nameCounter[word] = 1
        for word in item['unigram_text'].keys():
            if word in descCounter.keys():
                descCounter[word] += 1
            else:
                descCounter[word] = 1
    return  descCounter

def count_unigram(text):
    '''
    Count appearance number for each vocabulary
    '''
    counter = {}
    words = str(text).split()
    vocabs = set(words)
    for vocab in vocabs:
        if not vocab.isdigit():
            counter[vocab] = words.count(vocab)
    return counter


In [ ]:
newItems_wiki = []
for item in tqdm(corpus_split_df):
    newItem = combine_metadata_wiki(item)
    # newItem['unigram_title'] = count_unigram(newItem['title'])
    newItem['unigram_text'] = count_unigram(newItem['text'])
    newItems_wiki.append(newItem)

100%|██████████| 122/122 [00:00<00:00, 5700.05it/s]


In [ ]:
avgdl = sum(len(sentence) for sentence in corpus_split_df) / len(corpus_split_df)
def bm25(doc_len, corpus_len, doc_counter, corpus_counter, k=1.8,b=0.75):
    vector_len = len(corpus_counter)
    tfidf_vector = np.zeros((vector_len,))
    for i, key in enumerate(corpus_counter.keys()):
        if key in doc_counter.keys():
            tf = (k+1)*doc_counter[key]/(doc_counter[key]+k*(1-b+b*doc_len/avgdl) )
            idf = math.log((corpus_len-corpus_counter[key]+0.5)/(corpus_counter[key]+0.5)+1)
            tfidf_vector[i] = tf*idf
    return tfidf_vector

In [ ]:
avgdl

481.2049180327869

In [ ]:
textCounter = count_word_in_dataset_wiki(newItems_wiki)

In [ ]:
bm25_vectors = []
corpus_len = len(newItems_wiki)
for item in tqdm(newItems_wiki):
    doc_len = len(item['text'])
    bm25_vectors.append(
        bm25(doc_len, corpus_len, item['unigram_text'], textCounter)
    )

100%|██████████| 122/122 [00:00<00:00, 4032.89it/s]


In [ ]:
question="miễn học các môn học tiếng Anh"
new_qu=combine_metadata_wiki(question)
new_qu['unigram_text'] = count_unigram(new_qu['text'])
qu_bm25_vec=bm25(len(question),corpus_len,new_qu['unigram_text'],textCounter)

In [ ]:
qu_bm25_vec = np.reshape(qu_bm25_vec, (1,-1))
sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(qu_bm25_vec, bm25_vectors)
sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
idx = (-sim_maxtrix).argsort()[:5]
for _id in idx:
    print(_id, sim_maxtrix[_id])
    print(corpus_split_df[_id])

In [ ]:
# results = []
# for q in tqdm(questions):
#   test_question = q[0]
#   new_qu=combine_metadata_wiki(test_question)
#   new_qu['unigram_text'] = count_unigram(new_qu['text'])
#   qu_bm25_vec=bm25(len(test_question),corpus_len,new_qu['unigram_text'],textCounter)
#   qu_bm25_vec = np.reshape(qu_bm25_vec, (1,-1))
#   sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(qu_bm25_vec, bm25_vectors)
#   sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
#   idx = (-sim_maxtrix).argsort()[:5]
#   temp_result = []
#   for i in idx:
#     retrieval_score = sim_maxtrix[i]
#     result = qa_model({'question': test_question, 'context': corpus_split_df[i]})
#     result['retrieval_score'] = retrieval_score
#     result['predict_score'] = result['score']
#     result['context'] = corpus_split_df[i]
#     del result['score']
#     temp_result.append(result)
#   results.append({
#       'question': test_question,
#       'answers': temp_result,
#       'true_answer': q[1]
#   })

In [ ]:
result = []
for q in tqdm(questions):
    test_question = q[0]
    #Tokenize question in test data

    new_qu=combine_metadata_wiki(test_question)
    new_qu['unigram_text'] = count_unigram(new_qu['text'])
    qu_bm25_vec=bm25(len(test_question),corpus_len,new_qu['unigram_text'],textCounter)
    qu_bm25_vec = np.reshape(qu_bm25_vec, (1,-1))
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(qu_bm25_vec, bm25_vectors)
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    idx = (-sim_maxtrix).argsort()[:1]
    for _id in idx:
      predict = qa_model(question = test_question, context = corpus_split_df[_id])['answer']

    exact_match = compute_exact_match(predict,q[1])
    f1 = compute_f1(predict,q[1])
    result.append({'question': test_question, 'prediction': predict, 'truth': q[1], 'exact_match': exact_match, 'f1': f1})

In [ ]:
result = pd.DataFrame(result)
result

,question,prediction,truth,exact_match,f1
0,Chuẩn đầu ra tiếng Pháp?,DELF_B1 hoặc TCF_niveau_3.,DELF_B1 hoặc TCF_niveau_3,0,0.666667
1,Chuẩn đầu ra tiếng Nhật cho sinh viên hệ chính...,JLPT_N3,JLPT_N3,1,1.000000
2,Lớp học phần đại cương tối thiều nhiêu sinh viên?,50,50 sinh viên,0,0.500000
3,Nếu em vi phạm trong lúc thi thì xứ lý như thế...,hủy kết quả học tập trong lúc đang học).,xử lý kỷ luật,0,0.000000
4,Yêu cầu số tín chỉ tích lũy theo năm?,cầu.,thỏa điều kiện: (k-1) × 30 ≤ n < k × 30,0,0.000000
...,...,...,...,...,...
621,Lớp sinh viên được tổ chức như thế nào?,"theo khóa, khoa, chương trình, ngành hoặc chuy...",Lớp sinh viên: Lớp sinh viên được tổ chức the...,0,0.580645
622,Lớp sinh viên được giữ ổn định trong suốt các ...,"học tập, rèn luyện của sinh viên.",để triển khai các hoạt động có liên quan đến c...,0,0.461538
623,Mục đích của việc giữ ổn định lớp sinh viên tr...,"học tập, rèn luyện của sinh viên.",để triển khai các hoạt động có liên quan đến c...,0,0.461538
624,Các lớp học ngoại ngữ nằm trong chương trình đ...,20,20,1,1.000000


In [ ]:
sum(result['f1'])/625

0.37270351329836005